---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [10]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [61]:
import numpy as np 
import pandas as pd
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])
        #elif line[-1] == ':':
        #    town = line[:-1]
        #    state_town.append([state,town])
        #else:
        #    town = line[:line.index(',')]
        #    state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return df
''' with open('university_towns.txt','r') as file:
        text = file.readlines()
        state = []
        combine = []
        i = 0
        for line in text:
            if line[-7:] == '[edit]\n':
                state.append(line[:-7])     
                i = i + 1
            else:
                region = []
                region = [state[i-1] , line.split(' (')[0]]
                combine.append(region) 
        u_town = pd.DataFrame(combine, columns = ['State','RegionName']) 
    
        return u_town
'''   
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [60]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls' , skiprows = 7)
    df = df[['Unnamed: 4','Unnamed: 5']]
    index = df[df['Unnamed: 4'] == '2000q1'].index[0]
    df = df.iloc[index:]
    df.rename(columns = {'Unnamed: 4': 'Quarter','Unnamed: 5': 'GDP'}, inplace = True)
    lens = len(df)
    quarter = []
    for i in range((lens) - 2):
        if (df.iloc[i]['GDP']- df.iloc[i+1]['GDP'] > 0) &(df.iloc[i+1]['GDP']- df.iloc[i+2]['GDP'] > 0):
            quarter.append(df.iloc[i]['Quarter'])
    return quarter[0]
get_recession_start()

'2008q3'

In [77]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls' , skiprows = 7)
    df = df[['Unnamed: 4','Unnamed: 5']]
    index = df[df['Unnamed: 4'] == '2000q1'].index[0]
    df = df.iloc[index:]
    df.rename(columns = {'Unnamed: 4': 'Quarter','Unnamed: 5': 'GDP'}, inplace = True)
    lens = len(df)
    start= []
    end = []
    bottom = []
    recession = False
    for i in range(1, (lens) - 1):
        if not recession and (df.iloc[i-1]['GDP'] > df.iloc[i]['GDP'] >df.iloc[i+1]['GDP']):
            recession = True 
            start.append(df.iloc[i-1]['Quarter'])
        elif recession and (df.iloc[i-1]['GDP'] < df.iloc[i]['GDP'] < df.iloc[i+1]['GDP']):
            recession = False
            end.append((df.iloc[i+1]['Quarter']))
        elif recession and ((df.iloc[i-1]['GDP'] > df.iloc[i]['GDP']) & (df.iloc[i]['GDP'] < df.iloc[i+1]['GDP'])):
            recession = True
            bottom.append((df.iloc[i]['Quarter']))
    return(end[0])    
get_recession_end()

'2009q4'

In [78]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls' , skiprows = 7)
    df = df[['Unnamed: 4','Unnamed: 5']]
    index = df[df['Unnamed: 4'] == '2000q1'].index[0]
    df = df.iloc[index:]
    df.rename(columns = {'Unnamed: 4': 'Quarter','Unnamed: 5': 'GDP'}, inplace = True)
    lens = len(df)
    start= []
    end = []
    bottom = []
    recession = False
    for i in range(1, (lens) - 1):
        if not recession and (df.iloc[i-1]['GDP'] > df.iloc[i]['GDP'] >df.iloc[i+1]['GDP']):
            recession = True 
            start.append(df.iloc[i-1]['Quarter'])
        elif recession and (df.iloc[i-1]['GDP'] < df.iloc[i]['GDP'] < df.iloc[i+1]['GDP']):
            recession = False
            end.append((df.iloc[i+1]['Quarter']))
        elif recession and ((df.iloc[i-1]['GDP'] > df.iloc[i]['GDP']) & (df.iloc[i]['GDP'] < df.iloc[i+1]['GDP'])):
            recession = True
            bottom.append((df.iloc[i]['Quarter']))
    return(bottom[0])    
    #return '2009q2'
get_recession_bottom()    


'2009q2'

In [44]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    col = list(range(6, 15+12*3))
    #house_df.drop(house_df.columns[], axis = 1)
    #house_df
    house_df = house_df.drop(house_df.columns[[col]], axis = 1)
    house_df
    timeDict  = {}
    for i in range(2000,2017):
        for j in range(1,13):
            if 1<= j <= 3:
                after = 'q1'
                time = str(i)+'-0'+str(j)
                timeDict[time] = str(i)+after
            elif 4<=j<=6:
                after = 'q2'
                time = str(i)+'-0'+str(j)
                timeDict[time] = str(i)+after
            elif 7<=j<=9:
                after = 'q3'
                time = str(i)+'-0'+str(j)
                timeDict[time] = str(i)+after
            elif 10<=j<=12:
                after = 'q4'
                time = str(i)+'-'+str(j)
                timeDict[time] = str(i)+after            
    house_df = house_df.rename(columns = timeDict)
    time = []
    for i in range(2000,2017):
        for j in range(1,5):
            time.append(str(i)+'q'+str(j))
    time = time[:-1] 
    for item in time:
        temp = house_df[item].astype('float').mean(axis = 1)
        del house_df[item]
        house_df[item] = temp

    house_df = house_df.drop(house_df.columns[[0,3,4,5]], axis = 1)
    house_df = house_df.set_index(["State","RegionName"])
    house_df
    return house_df

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
CA,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
IL,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
PA,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
AZ,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
NV,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
CA,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
TX,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
CA,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [42]:
house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
house_df.ix[:,51:]

    

,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
5,131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
6,219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
7,85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
8,364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
9,88000.0,88800.0,89000.0,88900.0,89600.0,90600.0,91200.0,91700.0,92200.0,92600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [71]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (True, 0.005496427353694603, 'university town')
run_ttest()

(True, 0.005496427353694603, 'university town')